In [1]:
from vertexai.generative_models import GenerativeModel, Part
import vertexai
from google.cloud import storage
from pytube import YouTube #pip install pytube
import IPython.display 

vertexai.init(project="robotic-vista-415814", location="asia-northeast3")  

In [2]:
import os

def download_youtube(url): 
    yt = YouTube(url) # YouTube 객체 생성     
    stream = yt.streams.get_highest_resolution() # 가장 높은 해상도의 스트림 선택 
    # 현재 디렉터리에 동영상 다운로드 
    file_path = stream.download(output_path="./videos") 
    print("Download complete!") 
    return file_path 

def upload_to_gcs(bucket, file_path, file_name):
    # bucket에 파일을 업로드할 blob 객체 생성
    blob = bucket.blob(file_name)
    # blob 객체를 통해 로컬 파일을 버킷에 업로드
    blob.upload_from_filename(file_path)
    print("Upload complete!") 

def delete_video(bucket, file_name, file_path): 
    blob = bucket.blob(file_name)
    blob.delete() 
    if os.path.exists(file_path):
        os.remove(file_path) 
    print("Delete complete!")         

In [3]:
bucket_name = "robotic-vista-415814-bucket-01" 
bucket = storage.Client().bucket(bucket_name) 
url = "https://www.youtube.com/watch?v=7qQTyBW4uhI" 

In [4]:
bucket_name = "robotic-vista-415814-bucket-01"
bucket = storage.Client().bucket(bucket_name)
url = "https://www.youtube.com/watch?v=7qQTyBW4uhI" 

prompt = """
- 주인공과 영상의 배경을 소설처럼 디테일하게 묘사하세요.
- 주인공이 어느 도시에 있는지 말하고, 왜 그렇게 생각하는지 설명하세요.
- 만일 이 영상이 생성형 AI가 만들었다면, 어떤 부분이 가장 놀랍나요?
""" 

file_path = download_youtube(url)
file_name = os.path.basename(file_path)
upload_to_gcs(bucket, file_path, file_name)
video = Part.from_uri(
    uri=f"gs://{bucket_name}/{file_name}",
    mime_type="video/mp4", 
) 
contents = [prompt, video] 

model = GenerativeModel("gemini-1.5-flash-preview-0514") 
responses = model.generate_content(contents, stream=True) 
IPython.display.display(IPython.display.Video(file_path, width=800 ,embed=True)) 
responses = model.generate_content(contents, stream=True) 
for response in responses: 
    print(response.text.strip(), end="") 

print("\n\n") 
delete_video(bucket, file_name, file_path) 

Download complete!
Upload complete!
